1. 准备数据集，将数据集处理为目标组织结构
2. 加载数据集（定义数据处理类）构建一个获取数据集的类。记住，它派生自Dataset类，需要三个神奇的函数——__init__、__getitem__和__len__，它们始终被定义为：定义数据集加载器，设定批大小等。
3. 模型定义，或许可以加上损失函数定义优化方法定义
4. 训练代码
5. 测试代码
6. 推理代码
7. 准确率评估代码

In [2]:
# 引入依赖
from torchvision import datasets
import torch
import os
import skimage
import torchvision.datasets.mnist as mnist
import numpy
from pathlib import Path
import cv2
import shutil
from torch.utils.data import Dataset,DataLoader

In [4]:
# 准备数据
def prepare_data():
    '''要根据不同的需求整理数据的组织形式'''
    datasets.FashionMNIST(root = './data/FMNIST', download=True, train=False)
    root = "./data/Fashion-MNIST" 
    train_set = (
        mnist.read_image_file(os.path.join(root, 'train-images-idx3-ubyte')),
        mnist.read_label_file(os.path.join(root, 'train-labels-idx1-ubyte'))
    )
    test_set = (
        mnist.read_image_file(os.path.join(root,'t10k-images-idx3-ubyte')),
        mnist.read_label_file(os.path.join(root,'t10k-labels-idx1-ubyte'))
    )
    print("train set:", train_set[0].size())
    print("test set:", test_set[0].size())
    
    def convert_to_img(train=True):
        if(train):
            f = open(root + '/train.txt', 'w')
            data_path = root + '/train/'
            if(not os.path.exists(data_path)):
                os.makedirs(data_path)
            for i, (img, label) in enumerate(zip(train_set[0], train_set[1])):
                img_path = data_path + str(i) + '.jpg'
                cv2.imwrite(img_path, img.numpy())
                int_label = str(label).replace('tensor(', '')
                int_label = int_label.replace(')', '')
                f.write(str(i) + '.jpg' + ' ' + str(int_label) + '\n')
            f.close()
        else:
            f = open(root + '/test.txt', 'w')
            data_path = root + '/test/'
            if (not os.path.exists(data_path)):
                os.makedirs(data_path)
            for i, (img, label) in enumerate(zip(test_set[0], test_set[1])):
                img_path = data_path + str(i) + '.jpg'
                cv2.imwrite(img_path, img.numpy())
                int_label = str(label).replace('tensor(', '')
                int_label = int_label.replace(')', '')
                f.write(str(i) + '.jpg' + ' ' + str(int_label) + '\n')
            f.close()
    convert_to_img(True)
    convert_to_img(False)
    d = {'0': 'T-shirt',
    '1': 'Trouser',
    '2': 'Pullover',
    '3': 'Dress',
    '4': 'Coat',
    '5': 'Sandal',
    '6': 'Shirt',
    '7': 'Sneaker',
    '8': 'Bag',
    '9': 'Ankle boot',}
    with open('./data/Fashion-MNIST/test.txt', 'r') as rf:
        text=rf.read().split('\n')
        for i in range(len(text)):
            if text[i].strip():
                name,cla = text[i].split()
                # print(text[i].split())
                shutil.move('./data/Fashion-MNIST/test/'+name,'./data/Fashion-MNIST/test/{}/{}'.format(d[cla],name))

100%|██████████████████████████████████████████████████████████████████████████████| 26.4M/26.4M [00:33<00:00, 800kB/s]


Extracting ./data/FMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./data/FMNIST\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 132kB/s]


Extracting ./data/FMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./data/FMNIST\FashionMNIST\raw



100%|██████████████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:05<00:00, 839kB/s]


Extracting ./data/FMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./data/FMNIST\FashionMNIST\raw



100%|█████████████████████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<?, ?B/s]

Extracting ./data/FMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./data/FMNIST\FashionMNIST\raw



Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data/FMNIST
    Split: Test

In [ ]:
# 数据加载
# 主要是写一个类，然后写加载器，是pytorch中自带的        
class TrafficSign(Dataset):  
    def __init__(self, data_file, dataset_root):  
        self.transforms = [
            lambda x:x,
            transforms.RandomHorizontalFlip(),  # 左右翻转
            transforms.RandomVerticalFlip(),  # 垂直翻转
            transforms.RandomRotation((45,45)),   # 旋转45度
            transforms.RandomRotation((90,90)),   # 旋转45度
            transforms.RandomRotation((135,135)),   # 旋转45度
            transforms.RandomRotation((180,180)),   # 旋转45度
            transforms.RandomRotation((225,225)),   # 旋转45度
            transforms.RandomRotation((270,270)),   # 旋转45度
            transforms.RandomRotation((315,315)),   # 旋转45度
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            lambda x:transforms.Resize((x.size[0], x.size[1]//2))(x),   # 横向缩放
            lambda x:transforms.Resize((x.size[0]//2, x.size[1]))(x),   # 纵向缩放
            transforms.GaussianBlur(7, 3),  # 高斯模糊
            lambda x:Image.fromarray(iaa.MotionBlur(k=15, angle=0)(image = np.array(x)[:, :, ::-1] )),  # 运动模糊
            lambda x:transforms.Resize((30, 30))(transforms.ColorJitter(brightness=0.1, contrast=0.1)(x)), # 降低亮度并缩小
            ]
        with open(data_file,'r',encoding='utf-8') as rf:
            self.data = list()
            for pic in [i.split(',') for i in rf.read().split('\n') if i]:
                for j in range(len(self.transforms)):
                    if int(pic[1]) in [21,23,25,26,27,28,33,34,50,53,55] and j in [1]:
                        continue
                    self.data.append([pic[0]+'_{}'.format(j),pic[1]])
        self.dataset_root = dataset_root
  
    def __len__(self):  
        return len(self.data)
  
    def __getitem__(self, idx):  
        sample = self.data[idx]
        img = Image.open(self.dataset_root/Path(sample[0][:sample[0].rfind('_')])) # 打开图片，
        img = img.convert('RGB')   # 首先统一为RGB的形式，然后进行处理
        img = self.transforms[int(sample[0][sample[0].rfind('_')+1:])](img) # 数据增强处理  
        img = img.convert('L') # 改为单通道
        img = transforms.Resize((96, 96))(img) # 统一大小
        img = transforms.ToTensor()(img) 
        y = int(sample[1])
        return img,y

train_dataset = TrafficSign(data_root/'train_{}.csv'.format(cross_index),datasets_root)
train_iter = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset = TrafficSign(data_root/'test_{}.csv'.format(cross_index),datasets_root)
test_iter = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)

In [ ]:
# 模型定义GooGLeNet
import torch
from torch import nn
from d2l import torch as d2l
from pathlib import Path
from torchvision import datasets, transforms  
from sklearn.model_selection import train_test_split # pip install scikit-learn 
from torch.utils.data import Dataset, DataLoader  
from PIL import Image  
import json
from datetime import datetime
import argparse 
from torch.nn import functional as F
import time

class Animator:
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(14, 10),model_name='',index=''):
        """Defined in :numref:`sec_utils`"""
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
        self.model_name=model_name
        self.index = index

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        self.fig.savefig('log/log_{}_{}.png'.format(self.model_name,self.index))



class Inception(nn.Module):
    # `c1`--`c4` 是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最⼤汇聚层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)
    
b1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                    nn.ReLU(),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                    nn.ReLU(),
                    nn.Conv2d(64, 192, kernel_size=3, padding=1),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                    Inception(256, 128, (128, 192), (32, 96), 64),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                    Inception(512, 160, (112, 224), (24, 64), 64),
                    Inception(512, 128, (128, 256), (24, 64), 64),
                    Inception(512, 112, (144, 288), (32, 64), 64),
                    Inception(528, 256, (160, 320), (32, 128), 128),
                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                    Inception(832, 384, (192, 384), (48, 128), 128),
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten())




def train_ch6(train_iter,test_iter,num_epochs,lr,device,model_name,index):
    def init_weights(m):
        if type(m) == nn.Linear or type(m)==nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 62))
    net.apply(init_weights)
    print('train on', device)
    # print('模型yi转移到GPU',torch.cuda.memory_allocated()//1024//1024)
    net.to(device)
    # print('转移zyi完成',torch.cuda.memory_allocated()//1024//1024)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = Animator(xlabel='epoch',xlim=[1,num_epochs],legend=['train loss','train acc','test acc'],model_name=model_name,index=index)
    # timer,num_batches = d2l.Timer(),len(train_iter)
    num_batches = len(train_iter)
    best_acc = 0
    # print(num_batches)
    with open('log/log_{}_{}.log'.format(model_name,index),'w',encoding='utf-8') as wf:
        wf.write('')
    for epoch in range(num_epochs):
        start_time = time.perf_counter()
        out_str = '{}-cross_{}_epoch:{}/{}\t'.format(model_name,index,epoch,num_epochs)
        # print('epoch:{}'.format(epoch),end='\t')
        metric = d2l.Accumulator(3)
        net.train()
        for i,(X,y) in enumerate(train_iter):
            # timer.start()
            optimizer.zero_grad()
            # print('开始转移本batch的数据',torch.cuda.memory_allocated()//1024//1024)
            X,y = X.to(device),y.to(device)
            # print('本batch的数据转移完成',torch.cuda.memory_allocated()//1024//1024)
            y_hat = net(X)
            # print('本batch获得预测结果',torch.cuda.memory_allocated()//1024//1024)
            # print(y_hat.shape)
            # print(y_hat)
            # print(y.shape)
            # print(y)
            # quit()
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()
            with torch.no_grad():
                metric.add(l*X.shape[0],d2l.accuracy(y_hat,y),X.shape[0])
            # timer.stop()
            train_l = metric[0]/metric[2]
            train_acc = metric[1]/metric[2]
            if (i+1)%(num_batches//5)==0 or i==num_batches-1:
                animator.add(epoch+(i+1)/num_batches,(train_l,train_acc,None))
        test_acc = evaluate_accuracy_gpu(net,test_iter)
        if test_acc>best_acc:
            best_acc = test_acc
            weights_path = Path('./weights/')
            weights_path.mkdir(parents=True, exist_ok=True)
            if best_acc>0.0:
                torch.save(net.state_dict(),weights_path/Path('{}_{}_best_weights.pth'.format(model_name,index)))
        # torch.save(net.state_dict(), weights_path/Path('{}_{}_last_weights.pth'.format(model_name,index)))
        animator.add(epoch+1,(None,None,test_acc))
        with open('log/log_{}_{}.log'.format(model_name,index),'a',encoding='utf-8') as wf:
            wf.write(out_str+'\n')
        # print('本epoch结束',torch.cuda.memory_allocated()//1024//1024)
        process_time = time.perf_counter()-start_time
        out_str+=f'loss {train_l:.3f},train acc {train_acc:.3f},'f'test acc {test_acc:.3f},'f'time {process_time:.3f}s'
        print(out_str)

    
def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使⽤GPU计算模型在数据集上的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            # BERT微调所需的（之后将介绍）
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

def dataset(data_root):
    '''准备数据集
    '''
    data_root.mkdir(parents=True, exist_ok=True)
    # 检查是否存在训练集文件,没有则创建
    if not (data_root/Path('train_set')).exists():
        datasets.utils.download_and_extract_archive('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/train_set.zip',data_root,data_root)
    if not (data_root/Path('test_set')).exists():
        datasets.utils.download_and_extract_archive('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/test_set.zip',data_root,data_root)
    if not (data_root/Path('example.csv')).exists():
        datasets.utils.download_url('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/example.csv',data_root)

class TrafficSign(Dataset):  
    def __init__(self, datasets_root, data_root, data='train', lable=True):  
        self.data_type = data
        if data=='train':
            with open(data_root/Path('train.csv'),'r',encoding='utf-8') as rf:
                self.data = [i.split(',') for i in rf.read().split('\n')]
        elif data=='val':
            with open(data_root/Path('val.csv'),'r',encoding='utf-8') as rf:
                self.data = [i.split(',') for i in rf.read().split('\n')]
        elif data=='test':
            with open(data_root/Path('test.csv'),'r',encoding='utf-8') as rf:
                self.data = [[i,Path(i).name] for i in rf.read().split('\n')]
        else:
            assert '未知data类型：{}'.format(data) 
        with open(data_root/Path('classes.txt'),'r',encoding='utf-8') as rf:
            self.classes = rf.read().split(',')
        self.datasets_root = datasets_root
  
    def __len__(self):  
        return len(self.data)
  
    def __getitem__(self, idx):  
        sample = self.data[idx]
        img = Image.open(self.datasets_root/Path(sample[0]))
        # 图片处理
        img = img.convert('RGB')  
        # img = img.convert('L')
        resize = transforms.Resize((224, 224))  
        img = resize(img)  
        to_tensor = transforms.ToTensor()
        img = to_tensor(img) 
        if self.data_type == 'test':
            y = sample[1]
        else:
            y = self.classes.index(sample[1])
        return img,y
    
def test(test_loader,weights,device,model_name,index):
    net = nn.Sequential(b1, b2, b3, b4, b5, nn.Linear(1024, 62))
    net.load_state_dict(torch.load(weights,weights_only=True))
    net.to(device)
    net.eval()
    # print('test模型加载完毕',torch.cuda.memory_allocated()//1024//1024,torch.cuda.memory_reserved()//1024//1024)
    with open(data_root/Path('classes.txt'),'r',encoding='utf-8') as rf:
        classes = rf.read().split(',')
    with open(data_root/Path('classes.json'),'r',encoding='utf-8') as rf:
        classes_json=json.loads(rf.read())
    result = list()
    for data, name in test_loader:
        # print('test数据准备加载',torch.cuda.memory_allocated()//1024//1024,torch.cuda.memory_reserved()//1024//1024)
        data = data.to(device)
        # print('test数据加载完成',torch.cuda.memory_allocated()//1024//1024,torch.cuda.memory_reserved()//1024//1024)
        output=net(data)
        # print('test模型预测完成',torch.cuda.memory_allocated()//1024//1024,torch.cuda.memory_reserved()//1024//1024)
        predicted_class = torch.argmax(output,dim=1) 
        for i,j in zip(name,predicted_class.tolist()):
            result.append('{},{}'.format(i,classes_json[classes[j]]))
    with open('result/{}_cross_{}_{}.csv'.format(model_name,index,datetime.now().strftime("%Y-%m-%d_%H:%M:%S")),'w',encoding='utf-8') as wf:
        wf.write('ImageID,label\n')
        wf.write('\n'.join(result))
    print('测试完成',torch.cuda.memory_allocated()//1024//1024)

if __name__ == '__main__':
    # print('strat...',torch.cuda.memory_allocated()//1024//1024)
    datasets_root = Path('../data/traffic_sign/')
    data_root = Path('./data/traffic_sign/split_1/')
    dataset(datasets_root)
    batch_size = 208
    model_name = 'GoogLeNet-0004'
    parser = argparse.ArgumentParser(description='')  
    parser.add_argument('--index', type=str, help='交叉验证的划分号',default=0)
    args = parser.parse_args()
    cross_index = int(args.index)
    # print('加载训练集和验证集',torch.cuda.memory_allocated()//1024//1024)
    train_loader = DataLoader(dataset=TrafficSign(datasets_root,data_root,'train'),batch_size=batch_size,shuffle=True)
    val_loader = DataLoader(dataset=TrafficSign(datasets_root,data_root,'val'),batch_size=batch_size,shuffle=True)
    lr, num_epochs = 0.05, 80
    train_ch6( train_loader, val_loader, num_epochs, lr, d2l.try_gpu(), model_name, cross_index)
    print('开始测试',torch.cuda.memory_allocated()//1024//1024)
    test_loader = DataLoader(dataset=TrafficSign(datasets_root,data_root,'test'),batch_size=batch_size)
    test(test_loader,'weights/{}_{}_best_weights.pth'.format(model_name,cross_index),d2l.try_gpu(), model_name,cross_index)


In [ ]:
# m模型定义Alex模型
net = nn.Sequential(
    # 这⾥，我们使⽤⼀个11*11的更⼤窗⼝来捕捉对象。
    # 同时，步幅为4，以减少输出的⾼度和宽度。
    # 另外，输出通道的数⽬远⼤于LeNet
    nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减⼩卷积窗⼝，使⽤填充为2来使得输⼊与输出的⾼和宽⼀致，且增⼤输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使⽤三个连续的卷积层和较⼩的卷积窗⼝。
    # 除了最后的卷积层，输出通道的数量进⼀步增加。
    # 在前两个卷积层之后，汇聚层不⽤于减少输⼊的⾼度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这⾥，全连接层的输出数量是LeNet中的好⼏倍。使⽤dropout层来减轻过度拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这⾥使⽤Fashion-MNIST，所以⽤类别数为10，⽽⾮论⽂中的1000
    nn.Linear(4096, 62))

In [ ]:
# 模型训练
import torch
from torch import nn
from d2l import torch as d2l
from pathlib import Path
from torchvision import transforms  
from torch.utils.data import Dataset, DataLoader  
from PIL import Image  
import argparse 
import time
from models.ResNet import ResNet
import imgaug.augmenters as iaa
import numpy as np

class Animator:
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(14, 10),model_name='',index=''):
        """Defined in :numref:`sec_utils`"""
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
        self.model_name=model_name
        self.index = index

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        self.fig.savefig('log/log_{}_{}.png'.format(self.model_name,self.index))

def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使⽤GPU计算模型在数据集上的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            # BERT微调所需的（之后将介绍）
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


        
class TrafficSign(Dataset):  
    def __init__(self, data_file, dataset_root):  
        self.transforms = [
            lambda x:x,
            transforms.RandomHorizontalFlip(),  # 左右翻转
            transforms.RandomVerticalFlip(),  # 垂直翻转
            transforms.RandomRotation((45,45)),   # 旋转45度
            transforms.RandomRotation((90,90)),   # 旋转45度
            transforms.RandomRotation((135,135)),   # 旋转45度
            transforms.RandomRotation((180,180)),   # 旋转45度
            transforms.RandomRotation((225,225)),   # 旋转45度
            transforms.RandomRotation((270,270)),   # 旋转45度
            transforms.RandomRotation((315,315)),   # 旋转45度
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            transforms.RandomAffine(0,translate=(0.3,0.3)),  # 平移
            lambda x:transforms.Resize((x.size[0], x.size[1]//2))(x),   # 横向缩放
            lambda x:transforms.Resize((x.size[0]//2, x.size[1]))(x),   # 纵向缩放
            transforms.GaussianBlur(7, 3),  # 高斯模糊
            lambda x:Image.fromarray(iaa.MotionBlur(k=15, angle=0)(image = np.array(x)[:, :, ::-1] )),  # 运动模糊
            lambda x:transforms.Resize((30, 30))(transforms.ColorJitter(brightness=0.1, contrast=0.1)(x)), # 降低亮度并缩小
            ]
        with open(data_file,'r',encoding='utf-8') as rf:
            self.data = list()
            for pic in [i.split(',') for i in rf.read().split('\n') if i]:
                for j in range(len(self.transforms)):
                    if int(pic[1]) in [21,23,25,26,27,28,33,34,50,53,55] and j in [1]:
                        continue
                    self.data.append([pic[0]+'_{}'.format(j),pic[1]])
        self.dataset_root = dataset_root
  
    def __len__(self):  
        return len(self.data)
  
    def __getitem__(self, idx):  
        sample = self.data[idx]
        img = Image.open(self.dataset_root/Path(sample[0][:sample[0].rfind('_')])) # 打开图片，
        img = img.convert('RGB')   # 首先统一为RGB的形式，然后进行处理
        img = self.transforms[int(sample[0][sample[0].rfind('_')+1:])](img) # 数据增强处理  
        img = img.convert('L') # 改为单通道
        img = transforms.Resize((96, 96))(img) # 统一大小
        img = transforms.ToTensor()(img) 
        y = int(sample[1])
        return img,y
    
def process_bar(progress,total):
        # 计算当前进度条的百分比  
        percent = (progress / total) * 100  
        # 构造进度条字符串，使用空格来填充剩余部分  
        bar = f'[{">" * int(percent)}' + ' ' * (100 - int(percent)) + f']'  +' {}/{}'.format(progress,total)
        # 使用\r回到行首，然后打印进度条  
        print(f'\r{bar}', end='', flush=True)    

def init_weights(m):
    if type(m) == nn.Linear or type(m)==nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)

if __name__ == '__main__':
    device = d2l.try_gpu()
    print('train on {}'.format(device))
    # 首先加载数据，数据应该是csv的，存储样本和标签的信息
    print('load data...')
    datasets_root = Path('../data/traffic_sign/')
    data_root = Path('./data/traffic_sign_B/')
    # 确定是交叉验证的哪一个
    parser = argparse.ArgumentParser(description='')     
    parser.add_argument('--index', type=str, help='交叉验证的划分号',default=0)
    cross_index = int(parser.parse_args().index)
    ## 加载数据
    batch_size = 512
    train_dataset = TrafficSign(data_root/'train_{}.csv'.format(cross_index),datasets_root)
    train_iter = DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
    test_dataset = TrafficSign(data_root/'test_{}.csv'.format(cross_index),datasets_root)
    test_iter = DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)
    # num = 0
    # for i in test_iter:
    #     num+=i[0].shape[0]
    # print(num)
    # quit()
    # 构建模型
    print('generate model...')
    lr, num_epochs = 0.05, 5
    net  = ResNet
    net.apply(init_weights)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    # 开始训练
    print('start train...')
    model_name = 'ResNet-B-0002'
    animator = Animator(xlabel='epoch',xlim=[1,num_epochs],legend=['train loss','train acc','test acc'],model_name=model_name,index=cross_index)
    num_batches = len(train_iter)
    best_acc = 0
    with open('log/log_{}_{}.log'.format(model_name,cross_index),'w',encoding='utf-8') as wf:
        wf.write('')
    for epoch in range(num_epochs):
        start_time = time.perf_counter()
        out_str = '{}-cross_{}_epoch:{}/{}\t'.format(model_name,cross_index,epoch,num_epochs)
        metric = d2l.Accumulator(3)
        net.train()
        for i,(X,y) in enumerate(train_iter):
            optimizer.zero_grad()
            X,y = X.to(device),y.to(device)
            y_hat = net(X)
            l = loss(y_hat,y)
            l.backward()
            optimizer.step()
            optimizer.zero_grad()
            with torch.no_grad():
                metric.add(l*X.shape[0],d2l.accuracy(y_hat,y),X.shape[0])
            train_l = metric[0]/metric[2]
            train_acc = metric[1]/metric[2]
            if (i+1)%(num_batches//5)==0 or i==num_batches-1:
                animator.add(epoch+(i+1)/num_batches,(train_l,train_acc,None))
            process_bar(i,num_batches)
        test_acc = evaluate_accuracy_gpu(net,test_iter)
        if test_acc>best_acc:
            best_acc = test_acc
            weights_path = Path('./weights/')
            weights_path.mkdir(parents=True, exist_ok=True)
            if best_acc>0.9:
                torch.save(net.state_dict(),weights_path/Path('{}_{}_best_weights.pth'.format(model_name,cross_index)))
        # torch.save(net.state_dict(), weights_path/Path('{}_{}_last_weights.pth'.format(model_name,index)))
        animator.add(epoch+1,(None,None,test_acc))
        with open('log/log_{}_{}.log'.format(model_name,cross_index),'a',encoding='utf-8') as wf:
            wf.write(out_str+'\n')
        process_time = time.perf_counter()-start_time
        out_str+=f'loss {train_l:.3f},train acc {train_acc:.3f},'f'test acc {test_acc:.3f},'f'time {process_time:.3f}s'
        print(f'\r'+' '*120, end = '', flush=True)  
        print(f'\r'+out_str)  
    print(weights_path/Path('{}_{}_best_weights.pth'.format(model_name,cross_index)))


In [ ]:
# 模型测试
import torch
from torch import nn
from d2l import torch as d2l
from pathlib import Path
from torchvision import datasets, transforms  
from sklearn.model_selection import train_test_split # pip install scikit-learn 
from torch.utils.data import Dataset, DataLoader  
from PIL import Image  
import json
from datetime import datetime
import argparse 
from torch.nn import functional as F
import time

class Animator:
    """For plotting data in animation."""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(14, 10),model_name='',index=''):
        """Defined in :numref:`sec_utils`"""
        # Incrementally plot multiple lines
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # Use a lambda function to capture arguments
        self.config_axes = lambda: d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend)
        self.X, self.Y, self.fmts = None, None, fmts
        self.model_name=model_name
        self.index = index

    def add(self, x, y):
        # Add multiple data points into the figure
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        self.fig.savefig('log/log_{}_{}.png'.format(self.model_name,self.index))

class Residual(nn.Module): #@save
    def __init__(self, input_channels, num_channels,use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

def resnet_block(input_channels, num_channels, num_residuals,first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),nn.BatchNorm2d(64), nn.ReLU(),nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 2))
b4 = nn.Sequential(*resnet_block(128, 256, 2))
b5 = nn.Sequential(*resnet_block(256, 512, 2))

# net = nn.Sequential(b1, b2, b3, b4, b5,nn.AdaptiveAvgPool2d((1,1)),nn.Flatten(), nn.Linear(512, 10))

def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使⽤GPU计算模型在数据集上的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval() # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            # BERT微调所需的（之后将介绍）
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]



def split_dataset(datasets_root,data_root,exist_ok=True):
    test_path = datasets_root/Path('test_set_B')
    X_test = [item.relative_to(datasets_root) for item in test_path.rglob('*') if item.is_file()]
    with open(data_root/Path('test_B.csv'),'w',encoding='utf-8') as wf:
        wf.write('\n'.join([str(i) for i in X_test]))




def dataset(data_root):
    '''准备数据集
    '''
    data_root.mkdir(parents=True, exist_ok=True)
    # 检查是否存在训练集文件,没有则创建
    if not (data_root/Path('train_set')).exists():
        datasets.utils.download_and_extract_archive('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/train_set.zip',data_root,data_root)
    if not (data_root/Path('test_set')).exists():
        datasets.utils.download_and_extract_archive('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/test_set.zip',data_root,data_root)
    if not (data_root/Path('example.csv')).exists():
        datasets.utils.download_url('https://ai-contest-static.xfyun.cn/2024/%E4%BA%A4%E9%80%9A%E6%A0%87%E8%AF%86%E8%AF%86%E5%88%AB%E6%8C%91%E6%88%98%E8%B5%9B/example.csv',data_root)

class TrafficSign(Dataset):  
    def __init__(self, data_file,class_file, dataset_root,label=True):  
        if label:
            with open(data_file,'r',encoding='utf-8') as rf:
                self.data = [i.split(',') for i in rf.read().split('\n') if i]
        else:
            with open(data_file,'r',encoding='utf-8') as rf:
                self.data = [[i,Path(i).name] for i in rf.read().split('\n')] 
        with open(class_file,'r',encoding='utf-8') as rf:
            self.classes = rf.read().split(',')
        self.label = label
        self.dataset_root = dataset_root
  
    def __len__(self):  
        return len(self.data)
  
    def __getitem__(self, idx):  
        sample = self.data[idx]
        img = Image.open(self.dataset_root/Path(sample[0]))
        # 图片处理
        # img = img.convert('RGB')  
        img = img.convert('L')
        resize = transforms.Resize((96, 96))  
        img = resize(img)  
        to_tensor = transforms.ToTensor()
        img = to_tensor(img) 
        if not self.label:
            y = sample[1]
        else:
            y = self.classes.index(sample[1])
        return img,y

if __name__ == '__main__':
    device = d2l.try_gpu()
    print('test on {}'.format(device))
    # 首先加载数据，数据应该是csv的，存储样本和标签的信息
    datasets_root = Path('../data/traffic_sign/')
    data_root = Path('./data/traffic_sign/')
    batch_size = 512
    dataset = TrafficSign(data_file = data_root/'test.csv',class_file=data_root/'classes.txt',dataset_root=datasets_root)
    test_loader = DataLoader(dataset=dataset,batch_size=batch_size)
    #for i in test_loader:
    #    print(len(i),i[0].shape,i[1].shape)
    #    quit()
    # 加载模型和训练好的参数
    from models.ResNet import ResNet
    net  = ResNet
    weights = 'weights/ResNet-0000_0_best_weights.pth'
    net.load_state_dict(torch.load(weights,weights_only=True))
    net.to(device)
    net.eval()
    # 开始测试
    sample_num, right_num = 0,0
    for data, name in test_loader:
        data = data.to(device)
        output=net(data)
        predicted_class = torch.argmax(output,dim=1)
        for i,j in zip(name,predicted_class.tolist()):
            if i==j:
                right_num+=1
            sample_num+=1
    print('{}/{},{:.4f}'.format(right_num,sample_num,right_num/sample_num))



In [ ]:
# 准确度评价

In [ ]:
# 模型推理
import torch
from torch import nn
from d2l import torch as d2l
from pathlib import Path
from torchvision import datasets, transforms  
from sklearn.model_selection import train_test_split # pip install scikit-learn 
from torch.utils.data import Dataset, DataLoader  
from PIL import Image  
import json
from datetime import datetime
import argparse 
from torch.nn import functional as F
import time
import mimetypes
from models.ResNet import ResNet

class TrafficSign(Dataset):  
    def __init__(self, data_source, dataset_root):
        if data_source.is_file(): # 判断输入是不是文件
            if is_binary_file(data_source): # 如果是二进制文件，默认是输入一张图片
                pass
            else: # 如果不是二进制文件，默认是一个存储图片路径的文档
                with open(data_source,'r',encoding='utf-8') as rf:
                    self.data = [[i.split(',')[0],Path(i.split(',')[0]).name] for i in rf.read().split('\n') if i]
                # print(self.data)
        else: # 如果是一个文件夹，默认是存储图片的文件夹
            pass
        self.dataset_root = dataset_root
  
    def __len__(self):  
        return len(self.data)
  
    def __getitem__(self, idx):  
        sample = self.data[idx]
        img = Image.open(self.dataset_root/Path(sample[0]))
        # 图片处理
        img = img.convert('RGB')  
        # img = img.convert('L')
        resize = transforms.Resize((96, 96))  
        img = resize(img)  
        to_tensor = transforms.ToTensor()
        img = to_tensor(img) 
        y = sample[1]
        return img,y



def is_binary_file(file_path):
    mime_type, encoding = mimetypes.guess_type(file_path)
    return mime_type is None or mime_type.startswith('application/') or 'image' in mime_type


if __name__ == '__main__':
    device = d2l.try_gpu()
    print('infer on {}'.format(device))
    # 首先加载数据，数据应该是csv的，存储样本和标签的信息,也可以是一个文件夹
    data_source = Path('./data/traffic_sign/test_B.csv')
    datasets_root = Path('../data/traffic_sign/')
    data_root = Path('./data/traffic_sign/')
    batch_size = 512
    dataset = TrafficSign(data_source, datasets_root)
    test_loader = DataLoader(dataset=dataset,batch_size=batch_size)
    # 确定是交叉验证的哪一个
    parser = argparse.ArgumentParser(description='')  
    parser.add_argument('--index', type=str, help='交叉验证的划分号',default=0)
    cross_index = int(parser.parse_args().index)
    # 加载模型和训练好的参数\
    model_name = 'ResNet-B-0001'
    net  = ResNet
    weights = 'weights/{}_{}_best_weights.pth'.format(model_name,cross_index)
    net.load_state_dict(torch.load(weights,weights_only=True))
    net.to(device)
    net.eval()
    # 开始推理
    result = list()
    sample_num, right_num = 0,0
    for data, name in test_loader:
        data = data.to(device)
        output=net(data)
        predicted_class = torch.argmax(output,dim=1)
        for i,j in zip(name,predicted_class.tolist()):
            result.append([str(i),str(j)])
    with open('result/{}_cross_{}_{}.csv'.format(model_name,cross_index,datetime.now().strftime("%Y-%m-%d_%H-%M-%S")),'w',encoding='utf-8') as wf:
        wf.write('ImageID,label\n')
        for i in result:
            wf.write(','.join(i)+'\n')
    print('{} infer over'.format(cross_index))
